In [ ]:
pip install gradio

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import numpy as np
import gradio as gr
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
image_size = 256
train_datagen = ImageDataGenerator(rescale = 1/255,rotation_range = 10,horizontal_flip = True)
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Projects/Car Classification/Dataset/Train',target_size = (image_size,image_size),class_mode = "sparse")

In [ ]:
for image_batch,label_batch in train_generator:
    print(image_batch[0])
    break

In [ ]:
class_names = list(train_generator.class_indices.keys())
class_names

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1/255,rotation_range = 10,horizontal_flip = True)
test_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Projects/Car Classification/Dataset/Test',target_size = (image_size,image_size),class_mode = "sparse")

In [ ]:
for image_batch,label_batch in test_generator:
    print(image_batch[0])
    break

In [ ]:
size = 256
model = Sequential()
model.add(Convolution2D(32,(3,3),input_shape = (size,size,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Convolution2D(32,(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())

model.add(Dense(units = 96,activation = 'relu'))
model.add(Dense(units = 32,activation = 'relu'))
model.add(Dropout(0.30))
model.add(Dense(units = 7,activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator,validation_data = test_generator,epochs = 20)

In [ ]:
scores = model.evaluate(test_generator)

In [ ]:
scores

In [ ]:
history.history.keys()

In [ ]:
type(history.history['loss'])

In [ ]:
len(history.history['loss'])

In [ ]:
history.history['loss'][:10]

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
EPOCHS = 20
plt.figure(figsize = (8,8))

plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc,label = 'Training Accuracy')
plt.plot(range(EPOCHS),val_acc,label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training & Validation Accuracy')
plt.show()

plt.subplot(1,2,2)
plt.plot(range(EPOCHS),loss,label = 'Training Loss')
plt.plot(range(EPOCHS),val_loss,label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training & Validation Loss')
plt.show()

In [ ]:
model.save('Model.h5')

In [ ]:
model = load_model('Model.h5')

In [ ]:
def predict_image(img):
  img_2d = img.reshape(-1,256,256,3)
  prediction = model.predict(img_2d)[0]
  return {class_names[i] : float(prediction[i]) for i in range(7)}

In [ ]:
image = gr.inputs.Image(shape = (256,256))
label = gr.outputs.Label(num_top_classes = 7)
gr.Interface(fn = predict_image,inputs = image,outputs = label,interpretation = 'default').launch(share = True)